# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4978, uptime 0:00:33
mini_beamline                    RUNNING   pid 4979, uptime 0:00:33
random_walk                      RUNNING   pid 4980, uptime 0:00:33
random_walk_horiz                RUNNING   pid 4981, uptime 0:00:33
random_walk_vert                 RUNNING   pid 4982, uptime 0:00:33
simple                           RUNNING   pid 4983, uptime 0:00:33
thermo_sim                       RUNNING   pid 4984, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4985, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-23 19:57:07
Persistent Unique Scan ID: '0f5ad045-9143-4db7-86f3-2c4fe948659f'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 19:57:07.7 |     -1.000 |      98499 |


|         2 | 19:57:07.7 |     -0.500 |      97677 |
|         3 | 19:57:07.7 |      0.000 |      98577 |
|         4 | 19:57:07.7 |      0.500 |      98128 |
|         5 | 19:57:07.8 |      1.000 |      96625 |
+-----------+------------+------------+------------+


generator scan ['0f5ad045'] (scan num: 3)





('0f5ad045-9143-4db7-86f3-2c4fe948659f',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/0f5ad045-9143-4db7-86f3-2c4fe948659f-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-23 19:57:08
Persistent Unique Scan ID: '7aaf209a-4929-4153-a200-891fda8e315f'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 19:57:08.2 |          -1 | 1065246942 |


|         2 | 19:57:08.3 |          -0 | 1060910789 |


|         3 | 19:57:08.4 |           0 | 1055050943 |


|         4 | 19:57:08.6 |           0 | 1053486881 |


|         5 | 19:57:08.6 |           1 | 1053289963 |
+-----------+------------+-------------+------------+
generator scan ['7aaf209a'] (scan num: 4)





('7aaf209a-4929-4153-a200-891fda8e315f',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-23 19:57:09
Persistent Unique Scan ID: 'eca3207f-e06a-4360-8cb2-c0f297ed019c'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 19:57:09.2 | 1062125604 |


|         2 | 19:57:10.1 | 1119254804 |


|         3 | 19:57:11.1 | 1121098262 |


...

|        28 | 19:57:36.1 | 1064016028 |


|        29 | 19:57:37.1 | 1065689159 |


|        30 | 19:57:38.1 | 1121361052 |


+-----------+------------+------------+
generator count ['eca3207f'] (scan num: 5)





('eca3207f-e06a-4360-8cb2-c0f297ed019c',)